In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
sns.set_style('ticks')

# # 1 导入数据
data = pd.read_excel('./数据/Molecular_Descriptor.xlsx')
ADMET = pd.read_excel('./数据/ADMET.xlsx')
y = pd.read_excel('./数据/ERα_activity.xlsx')
X_fea = list(pd.read_excel('./数据/aa.xlsx')['features'][:20])
X1_fea = list(pd.read_excel('./数据/Caco-2-51个特征排序.xlsx')['features'][:20])
X2_fea = list(pd.read_excel('./数据/CYP3A4特征排序.xlsx')['features'][:20])
X3_fea = list(pd.read_excel('./数据/hERG特征排序.xlsx')['features'][:20])
X4_fea = list(pd.read_excel('./数据/HOB特征排序.xlsx')['features'][:20])
X5_fea = list(pd.read_excel('./数据/MN特征排序.xlsx')['features'][:20])
AMEDT_fea = X1_fea+X2_fea+X3_fea+X4_fea+X5_fea

# # 2 特征交集
def get_cross_fea(x1,x2):
    fea_count = {}
    for each in set(x1):
        if x1.count(each)>=3 and each in x2:
            fea_count[each] = x1.count(each)
return fea_count

if __name__ == "__main__":
    selected_fea = get_cross_fea(AMEDT_fea,X_fea)
# # 3 等级划分
pic = y['pIC50']
p1 = X_final[X_final['label']>=8.5].shape[0]/X_final.shape[0]
p2=X_final[(X_final['label']<8.5) & (X_final['label']>=7.5)].shape[0]/X_final.shape[0]
p3=X_final[(X_final['label']<7.5) & (X_final['label']>=5.5)].shape[0]/X_final.shape[0]
p4=X_final[(X_final['label']<5.5) & (X_final['label']>=4.5)].shape[0]/X_final.shape[0]
p5=X_final[X_final['label']<4.5].shape[0]/X_final.shape[0]

plt.figure(figsize=(12,8))
sns.distplot(X_final['label'])
plt.show()
plt.figure(figsize=(12,8))
sns.histplot(pic)
plt.show()

# # 4 按ADMET剔除化合物
ADMET.head()
ADMET[ADMET['hERG']==1].shape
ADMET_1 = ADMET[ADMET['hERG']==0]
def get_admetgood(x):
    indices = []
    for i in range(x.shape[0]):
        x1,x2,x3,x4 = x.iloc[i,1], x.iloc[i,2], x.iloc[i,4], x.iloc[i,5]
        if x1+x2+x3>=2:
            indices.append(x.index[i])
        elif x1+x2+x3 == 1 and x4==0:
            indices.append(x.index[i])
    return indices

if __name__ == '__main__':
    selected_sample = get_admetgood(ADMET_1)

# # 5 活性排序
X_final = data[selected_fea]
X_final = X_final.iloc[selected_sample]
X_final['label'] =  y['pIC50'].iloc[selected_sample]
X_vgood = X_final[X_final['label']>=7.5]
X_good = X_final[(X_final['label']<8.5) & (X_final['label']>=7.5)]
X_vgood = X_vgood.sort_values(by='label',ascending=False)
X_good = X_good.sort_values(by='label',ascending=False)

ADMET.iloc[X_vgood.index]['SMILES'].to_excel('./11个化合物.xlsx',index=False)
X_g = pd.concat([X_vgood,X_good],axis=0)
X_g.describe().iloc[:,:-1].to_excel('./数据描述.xlsx')
admet = ADMET.iloc[X_g.index][['Caco-2','CYP3A4','HOB','MN']]
zeros = []
ones = []
for col in admet.columns:
    zeros.append(admet[col].value_counts()[0])
    ones.append(admet[col].value_counts()[1])
all_set =[]
for i in range(admet.shape[0]):
    word = ''
    for j in admet.iloc[i,:]:
        word+=str(j)
    all_set.append(word)

compose = {}
for each in set(all_set):
    if each not in compose.keys():
        compose[each]=all_set.count(each)

plt.figure(figsize=(10,6))
sns.barplot(x=admet.columns,y = zeros,color='cornflowerblue',label=0)
sns.barplot(x=admet.columns,y = ones,bottom=zeros,color = 'lightsteelblue',label=1)
plt.ylabel('数量',fontsize=14,fontweight='bold')
plt.xlabel('性质',fontsize=14,fontweight='bold')
plt.legend(loc=1,fontsize=12)
plt.savefig('./47h个活性ADMET统计图.tif',dpi=100)
plt.show()


# # 看分布
cmap1 = sns.cubehelix_palette(start=0, light=1, as_cmap=True)
cmap2 = sns.cubehelix_palette(start=1, light=1, as_cmap=True)
cmap4 = sns.cubehelix_palette(start=-6, light=1, as_cmap=True)
cmap5 = sns.cubehelix_palette(start=30, light=1, as_cmap=True)
cmap6 = sns.cubehelix_palette(start=40, light=1, as_cmap=True)
cmap7 = sns.cubehelix_palette(start=6, light=1, as_cmap=True)
cmap8 = sns.cubehelix_palette(start=7, light=1, as_cmap=True)
cmap9 = sns.cubehelix_palette(start=50, light=1, as_cmap=True)

rs1 = np.random.RandomState(2)  
rs2 = np.random.RandomState(5)  
df1 = pd.DataFrame(rs1.randn(100,2)+2,columns = ['A','B'])
df2 = pd.DataFrame(rs2.randn(100,2)-2,columns = ['A','B'])
# 创建数据

sns.kdeplot(df1['A'],df1['B'],cmap = 'Greens', shade = True,shade_lowest=False)
sns.kdeplot(df2['A'],df2['B'],cmap = 'Blues', shade = True,shade_lowest=False)
plt.show()
sns.set_style('white')
plt.figure(figsize=(20,15))

plt.subplot(331)
sns.kdeplot(x='hmin', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap=cmap1)
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('hmin',fontsize=16,fontweight='bold')

plt.subplot(332)
sns.kdeplot(x='LipoaffinityIndex', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap=cmap2)
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('LipoaffinityIndex',fontsize=16,fontweight='bold')

plt.subplot(333)
sns.kdeplot(x='BCUTc-1h', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap=cmap3)
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('BCUTc-1h',fontsize=16,fontweight='bold')

plt.subplot(334)
sns.kdeplot(x='XLogP', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap='Reds')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('XLogP',fontsize=16,fontweight='bold')

plt.subplot(335)
sns.kdeplot(x='MDEC-23', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap='Blues')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('MDEC-23',fontsize=16,fontweight='bold')

plt.subplot(336)
sns.kdeplot(x='MDEC-33', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap='pink_r')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('MDEC-33',fontsize=16,fontweight='bold')

plt.subplot(337)
sns.kdeplot(x='ATSc3', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap='Greens')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('ATSc3',fontsize=16,fontweight='bold')

plt.subplot(338)
sns.kdeplot(x='minHBa', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15)
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('minHBa',fontsize=16,fontweight='bold')

plt.subplot(339)
sns.kdeplot(x='ATSc4', y='label',data=X_g, fill=True, cut=4, thresh=0, levels=15,cmap='magma_r')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('ATSc4',fontsize=16,fontweight='bold')
plt.tight_layout()
plt.savefig('./密度图.tif',dpi=100)
plt.show()
plt.figure(figsize=(20,15))

plt.subplot(331)
sns.distplot(X_g['hmin'],color='red')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('hmin',fontsize=16,fontweight='bold')

plt.subplot(332)
sns.distplot(X_g['LipoaffinityIndex'],color='blue')
plt.xlabel('LipoaffinityIndex',fontsize=16,fontweight='bold')

plt.subplot(333)
sns.distplot(X_g['BCUTc-1h'],color='gold')
plt.xlabel('BCUTc-1h',fontsize=16,fontweight='bold')

plt.subplot(334)
sns.distplot(X_g['XLogP'],color='green')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('XLogP',fontsize=16,fontweight='bold')

plt.subplot(335)
sns.distplot(X_g['MDEC-23'],color='gray')
plt.xlabel('MDEC-23',fontsize=16,fontweight='bold')

plt.subplot(336)
sns.distplot(X_g['MDEC-33'],color='purple')
plt.xlabel('MDEC-33',fontsize=16,fontweight='bold')

plt.subplot(337)
sns.distplot(X_g['ATSc3'],color='deeppink')
plt.ylabel('pIC50',fontsize=16,fontweight='bold')
plt.xlabel('ATSc3',fontsize=16,fontweight='bold')

plt.subplot(338)
sns.distplot(X_g['minHBa'],color='orange')
plt.xlabel('minHBa',fontsize=16,fontweight='bold')

plt.subplot(339)
sns.distplot(X_g['ATSc4'],color='teal')
plt.xlabel('ATSc4',fontsize=16,fontweight='bold')
plt.tight_layout()
plt.savefig('./分布图.tif',dpi=100)
plt.show()
